In [59]:
%cd /content/drive/MyDrive/Colab/CICIDS2017 serial

/content/drive/MyDrive/Colab/CICIDS2017 serial


In [2]:
!pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 647 kB/s 
     |████████████████████████████████| 3.3 MB 33.6 MB/s 
     |████████████████████████████████| 34.5 MB 13 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [4]:
#%% ART for TensorFlow v2 - Keras API
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import pandas as pd
import pickle as pkl
from sklearn.metrics import classification_report, precision_recall_fscore_support



from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod,ProjectedGradientDescent,DeepFool, CarliniL2Method
device = '/GPU:0'
if tf.__version__[0] != '2':
    raise ImportError('This notebook requires TensorFlow v2.')

#%% Load CICIDS2017 dataset
print('Loading CICIDS2017 dataset')
x_train = pd.read_csv('data/New_x_train.csv')
x_test = pd.read_csv('data/New_x_train.csv')
y_train = x_train.Label
y_test = x_test.Label

x_train = x_train.drop(columns=['Label'])
x_test = x_test.drop(columns=['Label'])


x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

# Exporting the clean examples in a .pkl file
with open("clean_train.pkl", "wb") as f:
  pkl.dump(x_train, f)
with open("clean_test.pkl", "wb") as f:
  pkl.dump(x_test, f)
#%% ### Define a neural network with 3 ReLU layers and a Softmax output

neuron_number = 256 # How many neuron for each hidden layer
output_number = 2 # How many output neurons
features_number = x_train.shape[1] # How many features in the input

model = tf.keras.Sequential(layers=[
    tf.keras.layers.Dense(neuron_number, input_shape=(features_number,), activation="relu"), # input = features_number, output = 256
    tf.keras.layers.Dense(neuron_number, activation="relu"), # input = 256 (implied), output = 256
    tf.keras.layers.Dense(neuron_number, activation="relu"), # input = 256 (implied), output = 256
    tf.keras.layers.Dense(output_number, activation="softmax") # input = 256 (implied), output = 2
])

### Train the model


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']);
model.summary()


# Iterrating on the dataset
best_value = 0.0 # Best evaluation metrics on test dataset
best_testing_model_file = "best_model_testing"
for i in range(3):
  print("Iteration {}, best value is {}".format(str(i), str(best_value)))
  model.fit(x_train, y_train, epochs=10, batch_size=100)
  eval_value = model.evaluate(x_test, y_test, verbose=2)[1]
  if eval_value > best_value:
    best_value = eval_value
    tf.keras.models.save_model(model, best_testing_model_file)

best_testing_model_file = "best_model_testing"
model = tf.keras.models.load_model(best_testing_model_file)
model.evaluate(x_test, y_test, verbose=2)
#%% Evaluate model accuracy on test data.
best_testing_model_file = "best_model_testing"
model = tf.keras.models.load_model(best_testing_model_file)
loss_test, accuracy_test = model.evaluate(x_test, y_test)
print('Accuracy on test data: {:4.2f}%'.format(accuracy_test * 100))
test_predictions = np.argmax(model.predict(x_test), axis=-1)
matrix = classification_report(y_true=y_test, y_pred=test_predictions)
print(matrix)
#%% Create a ART Keras classifier for the TensorFlow Keras model.
classifier = KerasClassifier(model=model, clip_values=(0, 1))

#%% Fast Gradient Sign Method attack
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.1)

# Generate adversarial test data.
x_test_adv = attack_fgsm.generate(x_test)
x_train_adv = attack_fgsm.generate(x_train)

# Evaluate accuracy on adversarial test data and calculate average perturbation.
loss_test, accuracy_test = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))
print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))
test_predictions = np.argmax(model.predict(x_test_adv), axis=-1)
matrix = classification_report(y_true=y_test, y_pred=test_predictions)
print("FGSM\n",matrix)

with open("New_FGSM_train.pkl", "wb") as f:
  pkl.dump(x_train_adv, f)
with open("New_FGSM_test.pkl", "wb") as f:
  pkl.dump(x_test_adv, f)

Loading CICIDS2017 dataset
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               16896     
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 2)                 514       
                                                                 
Total params: 148,994
Trainable params: 148,994
Non-trainable params: 0
_________________________________________________________________
Iteration 0, best value is 0.0
Train on 22731 samples
Epoch 1/10
22731/22731 [==============================] - 2s 81us/sample - loss: 0.2991 - accuracy: 0

In [6]:
#%% Projected Gradient descent
attack_pgd = ProjectedGradientDescent(estimator=classifier, eps=0.1,eps_step=0.01,verbose=False)

# Generate adversarial test data.
x_test_adv = attack_pgd.generate(x_test)
x_train_adv = attack_pgd.generate(x_train)

# Evaluate accuracy on adversarial test data and calculate average perturbation.
loss_test, accuracy_test = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))
print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))
test_predictions = np.argmax(model.predict(x_test_adv), axis=-1)
matrix = classification_report(y_true=y_test, y_pred=test_predictions)
print("PGD\n",matrix)


# x_train_adv_tf = tf.convert_to_tensor(x_train_adv, np.float32)
# x_test_adv_tf = tf.convert_to_tensor(x_test_adv, np.float32)
# Exporting the adversarial examples in a .pkl file
with open("New_PGD_train.pkl", "wb") as f:
  pkl.dump(x_train_adv, f)
with open("New_PGD_test.pkl", "wb") as f:
  pkl.dump(x_test_adv, f)

Accuracy on adversarial test data: 4.37%
Average perturbation: 0.05
PGD
               precision    recall  f1-score   support

         0.0       0.08      0.08      0.08     11365
         1.0       0.00      0.00      0.00     11366

    accuracy                           0.04     22731
   macro avg       0.04      0.04      0.04     22731
weighted avg       0.04      0.04      0.04     22731



In [7]:
attack_df = DeepFool(classifier=classifier)

# Generate adversarial test data.
x_test_adv = attack_df.generate(x_test)
x_train_adv = attack_df.generate(x_train)

# Evaluate accuracy on adversarial test data and calculate average perturbation.
loss_test, accuracy_test = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))
print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))
test_predictions = np.argmax(model.predict(x_test_adv), axis=-1)
matrix = classification_report(y_true=y_test, y_pred=test_predictions)
print("DF\n",matrix)


# x_train_adv_tf = tf.convert_to_tensor(x_train_adv, np.float32)
# x_test_adv_tf = tf.convert_to_tensor(x_test_adv, np.float32)
# Exporting the adversarial examples in a .pkl file
with open("New_DF_train.pkl", "wb") as f:
  pkl.dump(x_train_adv, f)
with open("New_DF_test.pkl", "wb") as f:
  pkl.dump(x_test_adv, f)

DeepFool:   0%|          | 0/22731 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/22731 [00:00<?, ?it/s]

Accuracy on adversarial test data: 12.75%
Average perturbation: 0.24
DF
               precision    recall  f1-score   support

         0.0       0.18      0.21      0.20     11365
         1.0       0.05      0.04      0.05     11366

    accuracy                           0.13     22731
   macro avg       0.12      0.13      0.12     22731
weighted avg       0.12      0.13      0.12     22731



In [8]:
##%% Carlini&Wagner Infinity-norm attack
attack_cw = CarliniL2Method(classifier=classifier,
                                        confidence=0.3,
                                        targeted=False,
                                        learning_rate=1,
                                        binary_search_steps=45,
                                        max_iter=10,
                                        initial_const=0.01,
                                        max_halving=5,
                                        max_doubling=5,
                                        batch_size=128,
                                        verbose=False)

# Generate adversarial test data.
x_test_adv = attack_cw.generate(x_test)
x_train_adv = attack_cw.generate(x_train)

# Evaluate accuracy on adversarial test data and calculate average perturbation.
loss_test, accuracy_test = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))
print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))
test_predictions = np.argmax(model.predict(x_test_adv), axis=-1)
matrix = classification_report(y_true=y_test, y_pred=test_predictions)
print("CW\n",matrix)


# x_train_adv_tf = tf.convert_to_tensor(x_train_adv, np.float32)
# x_test_adv_tf = tf.convert_to_tensor(x_test_adv, np.float32)
# Exporting the adversarial examples in a .pkl file
with open("New_CW_train.pkl", "wb") as f:
  pkl.dump(x_train_adv, f)
with open("New_CW_test.pkl", "wb") as f:
  pkl.dump(x_test_adv, f)

Accuracy on adversarial test data: 8.84%
Average perturbation: 0.04
CW
               precision    recall  f1-score   support

         0.0       0.11      0.11      0.11     11365
         1.0       0.07      0.07      0.07     11366

    accuracy                           0.09     22731
   macro avg       0.09      0.09      0.09     22731
weighted avg       0.09      0.09      0.09     22731



In [9]:
#%%   ### Create adversarial dataset for training

path = "" # If you saved your .pkl file in a specific path

# Clean examples with label 0
with open(path+"clean_train.pkl", "rb") as f:
  clean_train = pkl.load(f)
  clean_train = pd.DataFrame(clean_train)
  labels_0 = [0 for i in range(clean_train.shape[0])]
  clean_train[clean_train.shape[1]] = labels_0 

# Altered examples with label 1
labels_1 = [1 for i in range(clean_train.shape[0])]
with open(path+"New_FGSM_train.pkl", "rb") as f: # FGSM
  fgsm_dataset_train = pkl.load(f)
  fgsm_train_df = pd.DataFrame(fgsm_dataset_train)
  fgsm_train_df[clean_train.shape[1] - 1] = labels_1
with open(path+"New_PGD_train.pkl", "rb") as f: # PGD
  pgd_dataset_train = pkl.load(f)
  pgd_train_df = pd.DataFrame(pgd_dataset_train)
  pgd_train_df[clean_train.shape[1] - 1] = labels_1
with open(path+"New_CW_train.pkl", "rb") as f: # CW
  cw_dataset_train = pkl.load(f)
  cw_train_df = pd.DataFrame(cw_dataset_train)
  cw_train_df[clean_train.shape[1] - 1] = labels_1
with open(path+"New_DF_train.pkl", "rb") as f: # DF
  df_dataset_train = pkl.load(f)
  df_train_df = pd.DataFrame(df_dataset_train)
  df_train_df[clean_train.shape[1] - 1] = labels_1

# Create a balanced dataset containing all attacks
size_quarter = clean_train.shape[0] // 4
balanced_dataset_train = pd.DataFrame().append([
    fgsm_train_df.iloc[:size_quarter],
    pgd_train_df.iloc[size_quarter: 2*size_quarter],
    cw_train_df.iloc[2*size_quarter:3*size_quarter],
    df_train_df.iloc[3*size_quarter:],
])

# Create each dataset
fgsm_adv_dataset_train = clean_train.append(fgsm_train_df, ignore_index=True)
pgd_adv_dataset_train = clean_train.append(pgd_train_df, ignore_index=True)
cw_adv_dataset_train = clean_train.append(cw_train_df, ignore_index=True)
df_adv_dataset_train = clean_train.append(df_train_df, ignore_index=True)
balanced_adv_dataset_train = clean_train.append(balanced_dataset_train, ignore_index=True)


#%%     ### Create adversarial dataset for testing

path = "" # If you saved your .pkl file in a specific path

# Clean examples with label 0
with open(path+"clean_test.pkl", "rb") as f:
  clean_test = pkl.load(f)
  clean_test = pd.DataFrame(clean_test)
  labels = [0 for i in range(clean_test.shape[0])]
  clean_test[clean_test.shape[1]] = labels

# Altered examples with label 1
labels = [1 for i in range(clean_test.shape[0])]
with open(path+"New_FGSM_test.pkl", "rb") as f: # FGSM
  fgsm_dataset_test = pkl.load(f)
  fgsm_test_df = pd.DataFrame(fgsm_dataset_test)
  fgsm_test_df[clean_test.shape[1] - 1] = labels
with open(path+"New_PGD_test.pkl", "rb") as f: # PGD
  pgd_dataset_test = pkl.load(f)
  pgd_test_df = pd.DataFrame(pgd_dataset_test)
  pgd_test_df[clean_test.shape[1] - 1] = labels
with open(path+"New_CW_test.pkl", "rb") as f: # CW
  cw_dataset_test = pkl.load(f)
  cw_test_df = pd.DataFrame(cw_dataset_test)
  cw_test_df[clean_test.shape[1] - 1] = labels
with open(path+"New_DF_test.pkl", "rb") as f: # DF
  df_dataset_test = pkl.load(f)
  df_test_df = pd.DataFrame(df_dataset_test)
  df_test_df[clean_test.shape[1] - 1] = labels

# Create a balanced dataset containing all attacks
size_quarter = clean_test.shape[0] // 4
balanced_dataset_test = pd.DataFrame().append([
    fgsm_test_df.iloc[:size_quarter],
    pgd_test_df.iloc[size_quarter:2*size_quarter],
    cw_test_df.iloc[2*size_quarter:3*size_quarter],
    df_test_df.iloc[3*size_quarter:],
])

# Create each dataset
fgsm_adv_dataset_test = clean_test.append(fgsm_test_df, ignore_index=True)
pgd_adv_dataset_test = clean_test.append(pgd_test_df, ignore_index=True)
cw_adv_dataset_test = clean_test.append(cw_test_df, ignore_index=True)
df_adv_dataset_test = clean_test.append(df_test_df, ignore_index=True)
balanced_adv_dataset_test = clean_test.append(balanced_dataset_test, ignore_index=True)



In [10]:
TrainDataList =[fgsm_adv_dataset_train,pgd_adv_dataset_train,cw_adv_dataset_train,df_adv_dataset_train,balanced_adv_dataset_train]
TestDataList  =[fgsm_adv_dataset_test,pgd_adv_dataset_test,cw_adv_dataset_test,df_adv_dataset_test,balanced_adv_dataset_test]

s = (len(TrainDataList),len(TestDataList))
ResultsAD0=np.zeros(s)
ResultsAD1=np.zeros(s)
ResultsAD2=np.zeros(s)
ResultsFusionMVR=np.zeros(s)
ResultsFusionSBA=np.zeros(s)
ResultsFusionDS=np.zeros(s)
ResultsFusionDSWW=np.zeros(s)

for Itrain in range(len(TrainDataList)):
    
    x_adv_train = TrainDataList[Itrain].values[:, :-1]
    y_adv_train = TrainDataList[Itrain].values[:, -1]
    x_adv_test = TestDataList[Itrain].values[:, :-1]
    y_adv_test = TestDataList[Itrain].values[:, -1]
      
      ### Get the weights and config of each layer of the classification network
      
    weights_L0 = model.layers[0].get_weights()
    config_L0 = model.layers[0].get_config()
    weights_L1 = model.layers[1].get_weights()
    config_L1 = model.layers[1].get_config()
    weights_L2 = model.layers[2].get_weights()
    config_L2 = model.layers[2].get_config()
    
    
    
    ### Copy the layers config into new layers
    
    copy_L0_AD0 = tf.keras.layers.Dense.from_config(config_L0)
    copy_L0_AD1 = tf.keras.layers.Dense.from_config(config_L0)
    copy_L0_AD2 = tf.keras.layers.Dense.from_config(config_L0)
    copy_L1_AD1 = tf.keras.layers.Dense.from_config(config_L1)
    copy_L1_AD2 = tf.keras.layers.Dense.from_config(config_L1)
    copy_L2_AD2 = tf.keras.layers.Dense.from_config(config_L2)
    
    
    
    ### Create the ADs (Duplication + Fine-Tuning)
    
    neuron_number = 256
    output_number = 1
    
    
    AD0 = tf.keras.models.Sequential(layers=[
        copy_L0_AD0,
        tf.keras.layers.Dense(neuron_number, activation="relu"),
        tf.keras.layers.Dense(neuron_number, activation="relu"),
        tf.keras.layers.Dense(output_number, activation="sigmoid", name="output_layer")
    ])
    AD0.layers[0].set_weights(weights_L0)
  
    AD1 = tf.keras.models.Sequential(layers=[
        copy_L0_AD1,
        copy_L1_AD1,
        tf.keras.layers.Dense(neuron_number, activation="relu"),
        tf.keras.layers.Dense(neuron_number, activation="relu"),
        tf.keras.layers.Dense(output_number, activation="sigmoid", name="output_layer")
    ])
    AD1.layers[0].set_weights(weights_L0)
    AD1.layers[1].set_weights(weights_L1)
  
    AD2 = tf.keras.models.Sequential(layers=[
        copy_L0_AD2,
        copy_L1_AD2,
        copy_L2_AD2,
        tf.keras.layers.Dense(neuron_number, activation="relu"),
        tf.keras.layers.Dense(neuron_number, activation="relu"),
        tf.keras.layers.Dense(output_number, activation="sigmoid", name="output_layer")
    ])
    AD2.layers[0].set_weights(weights_L0)
    AD2.layers[1].set_weights(weights_L1)
    AD2.layers[2].set_weights(weights_L2)
      
    
    
    ### Train the ADs
    
    eval_metric = 'binary_accuracy'
    lr = 0.01
    optim0 = tf.keras.optimizers.Adam(learning_rate=lr)
    loss_fn0 = tf.keras.losses.BinaryCrossentropy()
    AD0.compile(optimizer=optim0,
                loss=loss_fn0,
                metrics=[eval_metric])
    optim1 = tf.keras.optimizers.Adam(learning_rate=lr)
    loss_fn1 = tf.keras.losses.BinaryCrossentropy()
    AD1.compile(optimizer=optim1,
                loss=loss_fn1,
                metrics=[eval_metric])
    optim2 = tf.keras.optimizers.Adam(learning_rate=lr)
    loss_fn2 = tf.keras.losses.BinaryCrossentropy()
    AD2.compile(optimizer=optim2,
                loss=loss_fn2,
                metrics=[eval_metric])
    
    # ---------------------------------------------------------------
    
    best_value = 0.0 # Best evaluation metrics on test dataset
    best_AD0_file = "AD0"
    
    # Iterrating on the dataset
    for i in range(3):
      print("AD0 Iteration {}".format(str(i)))
      AD0.fit(x=x_adv_train, y=y_adv_train, epochs=10, batch_size=100)
      eval_metrics = AD0.evaluate(x_adv_test, y_adv_test, verbose=2)
      eval_value = eval_metrics[AD0.metrics_names.index(eval_metric)]
      if eval_value > best_value:
        best_value = eval_value
        tf.keras.models.save_model(AD0, best_AD0_file)
    
    # -----------------------------------------------------------------
    
    best_value = 0.0 # Best evaluation metrics on test dataset
    best_AD1_file = "AD1"
    
    # Iterrating on the dataset
    for i in range(3):
      print("AD1 Iteration {}".format(str(i)))
      AD1.fit(x=x_adv_train, y=y_adv_train, epochs=10, batch_size=100)
      eval_metrics = AD1.evaluate(x_adv_test, y_adv_test, verbose=2)
      eval_value = eval_metrics[AD1.metrics_names.index(eval_metric)]
      if eval_value > best_value:
        best_value = eval_value
        tf.keras.models.save_model(AD1, best_AD1_file)
    
    # ------------------------------------------------------------------
    
    best_value = 0.0 # Best evaluation metrics on test dataset
    best_AD2_file = "AD2"
    
    # Iterrating on the dataset
    for i in range(3):
      print("AD2 Iteration {}".format(str(i)))
      AD2.fit(x=x_adv_train, y=y_adv_train, epochs=10, batch_size=100)
      eval_metrics = AD2.evaluate(x_adv_test, y_adv_test, verbose=2)
      eval_value = eval_metrics[AD2.metrics_names.index(eval_metric)]
      if eval_value > best_value:
        best_value = eval_value
        tf.keras.models.save_model(AD2, best_AD2_file)
  
    for Itest in range(len(TestDataList)):
        print(Itrain,Itest )

        x_adv_test = TestDataList[Itest].values[:, :-1]
        y_adv_test = TestDataList[Itest].values[:, -1]
         
        ### Evaluate the ADs
        
        AD0 = tf.keras.models.load_model("AD0")
        AD1 = tf.keras.models.load_model("AD1")
        AD2 = tf.keras.models.load_model("AD2")
        
        ResultsAD0[Itrain][Itest] = AD0.evaluate(x_adv_test, y_adv_test, verbose=2)[1]
        ResultsAD1[Itrain][Itest] = AD1.evaluate(x_adv_test, y_adv_test, verbose=2)[1]
        ResultsAD2[Itrain][Itest] = AD2.evaluate(x_adv_test, y_adv_test, verbose=2)[1]
        
        
        
        ## Fusion Rules
        
        
        
        ### Definition
        
        #### Majority Vote
        
        def MajorityVoteRule(ClassProbabilities):
          ClassFrequency = np.argmax(ClassProbabilities, axis=1) 
          counts = np.bincount(ClassFrequency)
          MajorityVotingClass = np.argmax(counts)
          return MajorityVotingClass
        
        #### Simple Bayes Average
        
        def SimpleBayesAverageFusion(ClassProbabilities):
          SimpleBayesAverage=sum(ClassProbabilities,0)/ClassProbabilities.shape[0]
          if SimpleBayesAverage[0]>=SimpleBayesAverage[1]:
            return 0,SimpleBayesAverage[0]
          return 1,SimpleBayesAverage[1]
        
        #### Dempster Shafer Evidence Combination
        
        def DempsterRule(m1, m2):
            ## extract the frame of discernment      
            sets=set(m1.keys()).union(set(m2.keys()))
            result=dict.fromkeys(sets,0)
            ## Combination process
            for i in m1.keys():
                for j in m2.keys():
                    if set(str(i)).intersection(set(str(j))) == set(str(i)):
                        result[i]+=m1[i]*m2[j]
                    elif set(str(i)).intersection(set(str(j))) == set(str(j)):
                        result[j]+=m1[i]*m2[j]
                         
            ## normalize the results
            f= sum(list(result.values()))
            for i in result.keys():
                result[i] /=f
            return result
        
        def DempsterShaferEvidenceCombinationRule (ClassProbabilities, Weights, DontKnowClass=False):
          MassFunctionList= []
          for i in range(ClassProbabilities.shape[0]):
            MassFunction = {"a":Weights[i,0]*ClassProbabilities[i,0], "b":Weights[i,1]*ClassProbabilities[i,1],"ab":1-Weights[i,0]*ClassProbabilities[i,0]-Weights[i,1]*ClassProbabilities[i,1]}
            MassFunctionList.append(MassFunction)
            
          for i,mass in enumerate(MassFunctionList):
            if i == 0:
              DSresult = mass
            else: 
              DSresult = DempsterRule(DSresult,mass)
        
          if DSresult['a'] >= DSresult['b']:
            return 0, DSresult['a']
          else:
            return 1, DSresult['b'] 
        
        
        ### Compute the results
        
        #### Turn mono-neuronal output into bi-class probabilities
        
        pred_AD0 = AD0.predict(x_adv_test).tolist()
        pred_AD1 = AD1.predict(x_adv_test).tolist()
        pred_AD2 = AD2.predict(x_adv_test).tolist()
        
        proba_AD0 = []
        for i in pred_AD0:
          proba_AD0.append([1-i[0], i[0]])
        
        proba_AD1 = []
        for i in pred_AD1:
          proba_AD1.append([1-i[0], i[0]])
          
        proba_AD2 = []
        for i in pred_AD2:
          proba_AD2.append([1-i[0], i[0]])
        
        #### Majority Vote
        
        guesses = []
        for i in range(len(proba_AD0)):
          class_proba = np.array([proba_AD0[i], proba_AD1[i], proba_AD2[i]])
          guesses.append(MajorityVoteRule(class_proba))
        
        correct = y_adv_test
        hit = 0
        for i in range(len(guesses)):
          if guesses[i] == correct[i]:
            hit += 1
        acc = hit / len(guesses)
        print(acc)
        ResultsFusionMVR[Itrain][Itest]=acc
        
        #### Simple Bayes Average
        
        guesses = []
        for i in range(len(proba_AD0)):
          class_proba = np.array([proba_AD0[i], proba_AD1[i], proba_AD2[i]])
          guesses.append(SimpleBayesAverageFusion(class_proba))
        
        correct = y_adv_test
        hit = 0
        for i in range(len(guesses)):
          if guesses[i][0] == correct[i]:
            hit += 1
        acc = hit / len(guesses)
        print(acc)
        
        ResultsFusionSBA[Itrain][Itest]=acc
        
        #### Dempster Shafer Evidence Combination
        
        # Get the weights
        weights = []
        correct = y_adv_test
        amount_1 = np.count_nonzero(correct == 1)
        amount_0 = np.count_nonzero(correct == 0)
        for proba_AD in [proba_AD0, proba_AD1, proba_AD2]:
          weight = []
          hit_0 = 0
          hit_1 = 0
          for i in range(len(proba_AD)):
            if proba_AD[i].index(max(proba_AD[i])) == correct[i]:
              if correct[i]:
                hit_1 += 1
              else:
                hit_0 += 1
          weight = [hit_0/amount_0, hit_1/amount_1]
          weights.append(weight)
        weights = np.array(weights)
        
        # #Delete these two lines
        # s=(3,2)
        # weights=np.ones(s)
        
        guesses = []
        for i in range(len(proba_AD0)):
          class_proba = np.array([proba_AD0[i], proba_AD1[i], proba_AD2[i]])
          guesses.append(DempsterShaferEvidenceCombinationRule(class_proba, weights))
        
        hit = 0
        for i in range(len(guesses)):
          if guesses[i][0] == correct[i]:
            hit += 1
        acc = hit / len(guesses)
        print(acc)
        
        ResultsFusionDS[Itrain][Itest]=acc
        
        
         #### Dempster Shafer Evidence Combination       without weight
         
        s=(3,2)
        weights=np.ones(s)*0.9999
        correct = y_adv_test
        guesses = []
        for i in range(len(proba_AD0)):
          class_proba = np.array([proba_AD0[i], proba_AD1[i], proba_AD2[i]])
          guesses.append(DempsterShaferEvidenceCombinationRule(class_proba, weights))
        
        hit = 0
        for i in range(len(guesses)):
          if guesses[i][0] == correct[i]:
            hit += 1
        acc = hit / len(guesses)
        print(acc)        
           
        
        ResultsFusionDSWW[Itrain][Itest]=acc

AD0 Iteration 0
Train on 45462 samples
Epoch 1/10
45462/45462 [==============================] - 3s 59us/sample - loss: 0.0142 - binary_accuracy: 0.9972
Epoch 2/10
45462/45462 [==============================] - 2s 54us/sample - loss: 0.0055 - binary_accuracy: 0.9989
Epoch 3/10
45462/45462 [==============================] - 3s 57us/sample - loss: 0.0016 - binary_accuracy: 0.9996
Epoch 4/10
45462/45462 [==============================] - 2s 54us/sample - loss: 0.0033 - binary_accuracy: 0.9996
Epoch 5/10
45462/45462 [==============================] - 3s 55us/sample - loss: 0.0272 - binary_accuracy: 0.9982
Epoch 6/10
45462/45462 [==============================] - 2s 52us/sample - loss: 0.0019 - binary_accuracy: 0.9997
Epoch 7/10
45462/45462 [==============================] - 3s 55us/sample - loss: 6.3590e-04 - binary_accuracy: 0.9998
Epoch 8/10
45462/45462 [==============================] - 3s 55us/sample - loss: 0.0013 - binary_accuracy: 0.9998
Epoch 9/10
45462/45462 [=====================

In [56]:
path=''
ResultsMatrix= np.zeros((5,4))

best_testing_model_file = "./best_model_testing"
model = tf.keras.models.load_model(best_testing_model_file)


with open(path+"clean_test.pkl", "rb") as f: # FGSM
  clean_test = pkl.load(f)
print(" Evaluation the model against clean examples")
pred_out=np.argmax(model.predict(clean_test, verbose=2),1)
ResultsMatrix[0,:]=list( precision_recall_fscore_support(y_true=y_test, y_pred=pred_out,average='weighted'))
ResultsMatrix[0,3] = model.evaluate(x=clean_test, y=y_test, verbose=2)[1]

with open(path+"New_FGSM_test.pkl", "rb") as f: # FGSM
  fgsm_dataset_test = pkl.load(f)
print("# Evaluating the model against the adversarial examples FGSM_test")
pred_out=np.argmax(model.predict(fgsm_dataset_test, verbose=2),1)
ResultsMatrix[1,:]=list( precision_recall_fscore_support(y_true=y_test, y_pred=pred_out,average='weighted'))
ResultsMatrix[1,3] = model.evaluate(x=fgsm_dataset_test, y=y_test, verbose=2)[1]

with open(path+"New_PGD_test.pkl", "rb") as f: # PGD
  pgd_dataset_test = pkl.load(f)
print("# Evaluating the model against the adversarial examples PGD_test")
pred_out=np.argmax(model.predict(pgd_dataset_test, verbose=2),1)
ResultsMatrix[2,:]=list( precision_recall_fscore_support(y_true=y_test, y_pred=pred_out,average='weighted'))
ResultsMatrix[2,3] = model.evaluate(x=pgd_dataset_test, y=y_test, verbose=2)[1]

with open(path+"New_CW_test.pkl", "rb") as f: # CW
  cw_dataset_test = pkl.load(f)
print("# Evaluating the model against the adversarial examples CW_test")
pred_out=np.argmax(model.predict(cw_dataset_test, verbose=2),1)
ResultsMatrix[3,:]=list( precision_recall_fscore_support(y_true=y_test, y_pred=pred_out,average='weighted'))
ResultsMatrix[3,3] = model.evaluate(x=cw_dataset_test, y=y_test, verbose=2)[1]

with open(path+"New_DF_test.pkl", "rb") as f: # DF
  df_dataset_test = pkl.load(f)

print("# Evaluating the model against the adversarial examples DF_test")
pred_out=np.argmax(model.predict(df_dataset_test, verbose=2),1)
ResultsMatrix[4,:]=list( precision_recall_fscore_support(y_true=y_test, y_pred=pred_out,average='weighted'))
ResultsMatrix[4,3] = model.evaluate(x=df_dataset_test, y=y_test, verbose=2)[1]

print(ResultsMatrix)


 Evaluation the model against clean examples
# Evaluating the model against the adversarial examples FGSM_test
# Evaluating the model against the adversarial examples PGD_test
# Evaluating the model against the adversarial examples CW_test
# Evaluating the model against the adversarial examples CW_test
[[0.96016128 0.957327   0.95726111 0.95732701]
 [0.1662396  0.24437992 0.19740097 0.24437992]
 [0.04070489 0.04368484 0.04213107 0.04368484]
 [0.08749984 0.08838151 0.08789403 0.08838151]
 [0.11664708 0.12753508 0.12129531 0.12753509]]


In [57]:
with open("ResultsAD0.pkl", "wb") as f:
  pkl.dump(ResultsAD0, f)
with open("ResultsAD1.pkl", "wb") as f:
  pkl.dump(ResultsAD1, f)
with open("ResultsAD2.pkl", "wb") as f:
  pkl.dump(ResultsAD2, f)
  
with open("ResultsFusionMVR.pkl", "wb") as f:
  pkl.dump(ResultsFusionMVR, f)
with open("ResultsFusionSBA.pkl", "wb") as f:
  pkl.dump(ResultsFusionSBA, f)
with open("ResultsFusionDS.pkl", "wb") as f:
  pkl.dump(ResultsFusionDS, f)
with open("ResultsFusionDSWW.pkl", "wb") as f:
  pkl.dump(ResultsFusionDSWW, f)

with open("ResultsMatrix.pkl", "wb") as f:
  pkl.dump(ResultsMatrix, f)   

In [60]:
print('\n ResultsAD0 ')
print(ResultsAD0)
print('\n ResultsAD1 ')
print(ResultsAD1)
print('\n ResultsAD2 ')
print(ResultsAD2)

print('\n ResultsFusionMVR ')
print(ResultsFusionMVR)
print('\n ResultsFusionSBA ')
print(ResultsFusionSBA)
print('\n ResultsFusionDS ')
print(ResultsFusionDS)
print('\n ResultsFusionDSWW ')
print(ResultsFusionDSWW)


print('\n ResultsMatrix ')
print(ResultsMatrix)   


 ResultsAD0 
[[0.99986804 0.99934012 0.53372049 0.79769915 0.83315736]
 [0.99991202 0.99991202 0.53972548 0.80526596 0.83647883]
 [0.97532004 0.98640621 0.95477539 0.7969293  0.92892963]
 [0.99986804 0.99986804 0.53330255 0.9748801  0.87686419]
 [0.99784434 0.99784434 0.93753022 0.96828121 0.97674984]]

 ResultsAD1 
[[0.99991202 0.99923015 0.53523821 0.7993049  0.83377326]
 [0.99989003 0.99989003 0.53712994 0.8007347  0.83504903]
 [0.97261447 0.96357399 0.95587522 0.7903524  0.92182481]
 [0.99901015 0.99901015 0.53563416 0.9766618  0.87730414]
 [0.99665654 0.99632663 0.91795343 0.96095639 0.96935904]]

 ResultsAD2 
[[0.99991202 0.99925214 0.54031938 0.8078835  0.83687472]
 [0.99989003 0.99989003 0.53583211 0.81336063 0.83751267]
 [0.95389557 0.95873475 0.94239146 0.8244468  0.92107695]
 [0.99423695 0.99423695 0.53519422 0.97320837 0.87354273]
 [0.99674451 0.99674451 0.92554224 0.96201223 0.97184461]]

 ResultsFusionMVR 
[[0.99991201 0.9993621  0.536558   0.8012186  0.83465312]
 [0.999